In [ ]:
import importlib.util

try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

try:
    import quimb
except ImportError:
    print("installing cirq-core[contrib]...")
    !pip install --quiet 'cirq-core[contrib]'
    print("installed cirq-core[contrib].")
        

# Cirq to Tensor Networks

Here we demonstrate turning circuits into tensor network representations of the circuit's unitary, final state vector, final density matrix, and final noisy density matrix. 

### Imports

In [ ]:
import cirq
import numpy as np
import pandas as pd
from cirq.contrib.svg import SVGCircuit

In [ ]:
import cirq.contrib.quimb as ccq
import quimb
import quimb.tensor as qtn

### Create a random circuit

In [ ]:
qubits = cirq.LineQubit.range(3)
circuit = cirq.testing.random_circuit(qubits, n_moments=10, op_density=0.8, random_state=52)
circuit = cirq.drop_empty_moments(circuit)
SVGCircuit(circuit)

### Circuit to Tensors
The circuit defines a tensor network representation. By default, the initial state is the `|0...0>` state (represented by the "zero qubit" operations labeled "Q0" in the legend. "Q1" are single qubit operations and "Q2" are two qubit operations. The open legs are the indices into the state vector and are of the form "i{m}_q{n}" where `m` is the time index (given by the returned `qubit_frontier` dictionary) and "n" is the qubit string.

Note: this notebook relies on unreleased Cirq features. If you want to try these features, make sure you install cirq via `pip install cirq --pre`.

In [ ]:
tensors, qubit_frontier, fix = ccq.circuit_to_tensors(circuit, qubits)
tn = qtn.TensorNetwork(tensors)
print(qubit_frontier)
from matplotlib import pyplot as plt
tn.graph(fix=fix, color=['Q0', 'Q1', 'Q2'], figsize=(8,8))

### To dense

In [ ]:
psi_tn = ccq.tensor_state_vector(circuit, qubits)
psi_cirq = cirq.final_state_vector(circuit, qubit_order=qubits)
np.testing.assert_allclose(psi_cirq, psi_tn, atol=1e-7)

### Circuit Unitary
We can also leave the input legs open which gives a tensor network representation of the unitary

In [ ]:
tensors, qubit_frontier, fix = ccq.circuit_to_tensors(circuit, qubits, initial_state=None)
tn = qtn.TensorNetwork(tensors)
print(qubit_frontier)
tn.graph(fix=fix, color=['Q0', 'Q1', 'Q2'], figsize=(8, 8))

### To dense

In [ ]:
u_tn = ccq.tensor_unitary(circuit, qubits)
u_cirq = circuit.unitary(qubit_order=qubits)
np.testing.assert_allclose(u_cirq, u_tn, atol=1e-7)

### Density Matrix
We can also turn a circuit into its density matrix. The density matrix resulting from the evolution of the `|0><0|` initial state can be thought of as two copies of the circuit: one going "forwards" and one going "backwards" (i.e. use the complex conjugate of each operation). Kraus operator noise operations "link" the forwards and backwards circuits. As such, the density matrix for pure states is simple.

Note: for density matrices, we return a `fix` variable for a circuit-like layout of the tensors when calling `tn.graph`.

In [ ]:
tensors, qubit_frontier, fix = ccq.circuit_to_density_matrix_tensors(circuit=circuit, qubits=qubits)
tn = qtn.TensorNetwork(tensors)
tn.graph(fix=fix, color=['Q0', 'Q1', 'Q2'])

### Noise
Noise operations entangle the forwards and backwards evolutions. The new tensors labeled "kQ1" are 1-qubit Kraus operators.

In [ ]:
noise_model = cirq.ConstantQubitNoiseModel(cirq.DepolarizingChannel(p=1e-3))
circuit = cirq.Circuit(noise_model.noisy_moments(circuit.moments, qubits))
SVGCircuit(circuit)

In [ ]:
tensors, qubit_frontier, fix = ccq.circuit_to_density_matrix_tensors(circuit=circuit, qubits=qubits)
tn = qtn.TensorNetwork(tensors)
tn.graph(fix=fix, color=['Q0', 'Q1', 'Q2', 'kQ1'], figsize=(8,8))

### For 6 or fewer qubits, we specify the contraction ordering.
For low-qubit-number circuits, a reasonable contraction ordering is to go in moment order (as a normal simulator would do). Otherwise, quimb will try to find an optimal ordering which was observed to take longer than it takes to do the contraction itself. We show how to tell quimb to contract in order by using the moment tags.

In [ ]:
partial = 12
tags_seq = [(f'i{i}b', f'i{i}f') for i in range(partial)]
tn.graph(fix=fix, color = [x for x, _ in tags_seq] + [y for _, y in tags_seq], figsize=(8, 8))

### The result of a partial contraction

In [ ]:
tn2 = tn.contract_cumulative(tags_seq, inplace=False)
tn2.graph(fix=fix, color=['Q0', 'Q1', 'Q2', 'kQ1'], figsize=(8, 8))

### To Dense

In [ ]:
rho_tn = ccq.tensor_density_matrix(circuit, qubits)
rho_cirq = cirq.final_density_matrix(circuit, qubit_order=qubits)
np.testing.assert_allclose(rho_cirq, rho_tn, atol=1e-5)

## Profile
For low-qubit-number, deep, noisy circuits, the quimb contraction is faster.

In [ ]:
import timeit

def profile(n_qubits: int, n_moments: int):
    qubits = cirq.LineQubit.range(n_qubits)
    circuit = cirq.testing.random_circuit(qubits, n_moments=n_moments, op_density=0.8)
    noise_model = cirq.ConstantQubitNoiseModel(cirq.DepolarizingChannel(p=1e-3))
    circuit = cirq.Circuit(noise_model.noisy_moments(circuit.moments, qubits))
    circuit = cirq.drop_empty_moments(circuit)
    n_moments = len(circuit)
    variables = {'circuit': circuit, 'qubits': qubits}

    setup1 = [
        'import cirq',
        'import numpy as np',
    ]
    n_call_cs, duration_cs = timeit.Timer(
        stmt='cirq.final_density_matrix(circuit)',
        setup='; '.join(setup1),
        globals=variables).autorange()

    setup2 = [
        'from cirq.contrib.quimb import tensor_density_matrix',
        'import numpy as np',
    ]
    n_call_t, duration_t = timeit.Timer(
        stmt='tensor_density_matrix(circuit, qubits)',
        setup='; '.join(setup2),
        globals=variables).autorange()

    return {
        'n_qubits': n_qubits,
        'n_moments': n_moments,
        'duration_cirq': duration_cs,
        'duration_quimb': duration_t,
        'n_call_cirq': n_call_cs,
        'n_call_quimb': n_call_t,
    }

In [ ]:
records = []
max_qubits = 6
max_moments = 500
for n_qubits in [3, max_qubits]:
    for n_moments in range(1, max_moments, 50):
        record = profile(n_qubits=n_qubits, n_moments=n_moments)
        records.append(record)
        print('.', end='', flush=True)

df = pd.DataFrame(records)
df.head()

In [ ]:
def select(df, k, v):
    return df[df[k] == v].drop(k, axis=1)

pd.DataFrame.select = select

def plot1(df, labelfmt):
    for k in ['duration_cirq', 'duration_quimb']:
        plt.plot(df['n_moments'], df[k], '.-', label=labelfmt.format(k))
    plt.legend(loc='best')


def plot(df):
    df['duration_cirq'] /= df['n_call_cirq']
    df['duration_quimb'] /= df['n_call_quimb']
    plot1(df.select('n_qubits', 3), 'n = 3, {}')
    plot1(df.select('n_qubits', 6), 'n = 6, {}')
    plt.xlabel('N Moments')
    plt.ylabel('Time / s')
    
plot(df)
plt.tight_layout()